# ANÁLISIS DE SENTIMIENTOS APLICADO A LOS TEXTOS

En este cuaderno se va a explicar como se han realizado los diferentes análisis de sentimiento a los textos mediante los códigos de Python.
Digo diferentes porque se han realizado tres análisis con tres librerías diferentes.

Estas librerías son:<br>

**__TextBlob__**: Es una librería para el procesamiento de texto o muy conocida, usa NLTK que es otra librería para el procesamiento del lenguaje natural, y además versátil, que cuenta con traductor propio, pero como es una librería que los módulos de análisis de sentimiento está  especializado en el inglés, se utilizará el texto en inglés. TextBlob general el análisis de sentimiento teniendo en cuenta algoritmos basados en tokenización y en lexicon, los lexicon son diccionarios con ponderación de palabras diferenciándose en positivas y negativas en este caso.
Los resultados de este análisis se representan en los campos de textblob_polarity que contiene la polaridad de los textos enteros, es decir, como de positivos o de negativos son, un texto con polaridad 1 es positivo y con polaridad -1 es negativo.
Y en el campo textblob_subjetivity  que representa la subjetividad del texto, cuanto más alto sea el valor, más cerca esta de ser una hecho o una verdad universal, y cuanto más bajo más tiende a ser una opinión. Estos campos se representan en valores.

**__VADER__**: Esta librería también esta basada en los textos en inglés. Sus iniciales corresponden a  Valence Aware Dictionary and Sentiment Reasoner. También es un algoritmo basado en un lexicon pero con una mecánica y valores diferentes a la hroa de conformar las frases, teniendo en cuenta signos de puntuación y creando n-gramas de palabras. Los resultados se traducen en un porcentaje de como de negativo, positivo o nautral es, además opera y crea el dato compound que señala de -1 a 1 cuanto de positivo es el texto. En el caso de este proyecto se recoge el valor de compound.

**__pysentimiento__**: Está librería la encuento de especial interés para este trabajo ya que es una librería para el procesamiento del lenguaje natural que se especializa en el idioma español y dispone de alternativas muy interesantes como son el análisis de sentimiento, que dice si es positivo, neutro  o negativo, el análisis de emociones que indica si hay tristeza, alegría, enfado u otras emociones que se puede traducir en neutralidad, e incluso un detector de discurso de odio que se ha pasado.




Las primeras librerias usaran textos en ingles, mientras que la última es una librería especializada en textos en español.


# PSEUDOCÓDIGOS

Los pseudocódigos son muy sencillos por lo que se usará un pesudocódigo para los tres aunque luego se codifiquen por separado.

Se conecta a la base de datos<br>
Se recupera la tabla<br>
Se extraen los textos en fila<br>
Para textblob: Calcular y recoger polaridad y subjetividad<br>
Para vader: Calcular y recoger la media o valor compuesto<br>
Para pysentimiento = Recoger sentimientos, emociones y discurso de odio<br>

Añadir columnas a la tabla<br>

Con esto, se empieza a codificar, una librería por una. El proceso de insercción de columnas no se muestra en este cuaderno.

# TEXTBLOB

Primero se importan las librerías y se hace la conexión a la base de datos

In [ ]:
import mysql.connector
import codecs
import pandas as pd
from nltk.tokenize import TweetTokenizer
import nltk
#nltk.download('stopwords')
tweet_tokenizer = TweetTokenizer()
from nltk.corpus import stopwords
import string
from textblob import TextBlob
from tqdm import tqdm

codecs.register(lambda name: codecs.lookup('utf8') if name == 'utf8mb4' else None)

database = mysql.connector.connect(
    host= "localhost",
    user= "root",
    passwd= "",
    database= "TFG",
    use_unicode = True, 
    charset="utf8mb4",
    collation = "utf8mb4_general_ci"
)
database.set_charset_collation("utf8mb4","utf8mb4_general_ci")

cursor= database.cursor()

Acto seguido se codifica la función que calculará el sentimiento con esta librería

In [ ]:
tabla = "tweets_" # se escoge tabla 
columna_texto = "texto_ingles" # se escoge columna de textos

def textblob (tabla,columna_texto):
    df = pd.read_sql(f"SELECT * FROM {tabla}",database) # se importan datos


    sql = f"UPDATE {tabla} SET textblob_polaridad = %s WHERE {columna_texto} = %s"# se crea columna
    sql2 = f"UPDATE {tabla} SET textblob_subjetividad = %s WHERE {columna_texto} = %s"# se crean columna

    df = df[~df[columna_texto].isnull()]
    print(df.info())
    texto_en = df[columna_texto].tolist()

    pol_list = []
    subj_list = []
    for a in tqdm(range(len(texto_en))): # se calcula la polaridad y la subjetividad
        men = TextBlob(texto_en[a])
        pol = float(men.sentiment.polarity)
        pol_list.append(pol)
        subj = float(men.sentiment.subjectivity)
        subj_list.append(subj)

    df["texblob_polaridad"] = pol_list
    df["textblob_subjetividad"] = subj_list

# VADER

Primero se importan las librerías y se hace la conexion a la base de datos

In [ ]:
import mysql.connector
import codecs
import pandas as pd
from nltk.tokenize import TweetTokenizer
import nltk
#nltk.download('stopwords')
tweet_tokenizer = TweetTokenizer()
from nltk.corpus import stopwords
import string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tqdm

codecs.register(lambda name: codecs.lookup('utf8') if name == 'utf8mb4' else None)

database = mysql.connector.connect(
    host= "localhost",
    user= "root",
    passwd= "",
    database= "TFG",
    use_unicode = True, 
    charset="utf8mb4",
    collation = "utf8mb4_general_ci"
)
database.set_charset_collation("utf8mb4","utf8mb4_general_ci")

cursor= database.cursor()


Acto seguido se codifica la función que calcula los valores paraa los textos

In [ ]:
tabla = "tweets"
columna_texto = "texto_ingles"


def textblob (tabla,columna_texto):
    import pandas as pd
    analyzer = SentimentIntensityAnalyzer() # se crea el analizador de textos
    df = pd.read_sql(f"SELECT * FROM {tabla}",database)



    df = df[~df[columna_texto].isnull()] # se eliminan textos nulos
    print(df.info())
    texto_en = df[columna_texto].tolist()

    comp_list = []
    for a  in tqdm(range(len(texto_en))):# se calcula el valor y se extare la media
        vs = analyzer.polarity_scores(texto_en[a])
        comp = vs["compound"]
        comp_list.append(comp)
    df["vader"] = comp_list

# PYSENTIMIENTO

Se empieza importando a librerías y creando la conexión a la base de datos

In [ ]:
import mysql.connector
import codecs
import pandas as pd
from nltk.tokenize import TweetTokenizer

import string
from textblob import TextBlob
from pysentimiento import create_analyzer
from pysentimiento.preprocessing import preprocess_tweet
from tqdm import tqdm

codecs.register(lambda name: codecs.lookup('utf8') if name == 'utf8mb4' else None)

database = mysql.connector.connect(
    host= "localhost",
    user= "root",
    passwd= "",
    database= "TFG",
    use_unicode = True, 
    charset="utf8mb4",
    collation = "utf8mb4_general_ci"
)
database.set_charset_collation("utf8mb4","utf8mb4_general_ci")

cursor= database.cursor()

Se continua ejecutando la función que identificara los sentimientos, la emocion y el odio de los textos

In [ ]:
tabla = "tweets"
columna_texto = "texto_espanol"
def textblob (tabla,columna_texto):
    try: # se inserta las columnas si hace falta
        sqlinsert = f"ALTER TABLE {tabla} ADD pysen_sentimiento VARCHAR(4)"
        cursor.execute(sqlinsert)
        sqlinsert2 = f"ALTER TABLE {tabla} ADD pysen_emocion VARCHAR(10)"
        cursor.execute(sqlinsert2)
        sqlinsert3 = f"ALTER TABLE {tabla} ADD pysen_odio VARCHAR(50)"
        cursor.execute(sqlinsert3)
    except:
        pass
    df = pd.read_sql(f"SELECT * FROM {tabla}",database)# se lee la base de datps


    sql = f"UPDATE {tabla} SET pysen_sentimiento = %s WHERE {columna_texto} = %s"
    sql2 = f"UPDATE {tabla} SET pysen_emocion = %s WHERE {columna_texto} = %s"
    sql3 = f"UPDATE {tabla} SET pysen_odio = %s WHERE {columna_texto} = %s"

    df = df[~df[columna_texto].isnull()]
    print(df.info())
    texto_en = df[columna_texto].tolist()# se eliminan valores nulos si los hay

    sentimiento = create_analyzer(task="sentiment", lang="es")# se crea el analizador de sentimientos
    emocion = create_analyzer(task="emotion", lang="es")# se crea el analizador de emociones
    odio = create_analyzer(task="hate_speech", lang="es") # se crea el detector de odio
    odio_l = []
    emo_l = []
    sent_l = []
    for a in tqdm(range(len(texto_en))): # se calcula y se recoge los resultados

        pre = preprocess_tweet(texto_en[a])
        sent = sentimiento.predict(pre).output
        emo = emocion.predict(pre).output
        odi =" | ".join(odio.predict(pre).output)

        odio_l.append(odi)
        emo_l.append(emo)
        sent_l.append(sent)
        
        
    df["pysen_sentimiento"] = sent_l
    df["pysen_emocion"] = emo_l
    df["pysen_odio"] = odio_l
   

 # CONCLUSIONES

Después de analaizar el texto con las tres librerías, la más rápida fue textblob, pero es la que se descarta debido a su alto porcentaje de error. <br<
Vader analiza bien los textos con un porcentaje menor de error.<br>
pysentimiento es la librería más compleja y por ello lo que más tarda, teniendo una diferencia de 4h con la de vader, qeu tarda 30min en analizar 1,2M de textos.